In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
import matplotlib.pyplot as plt

In [3]:
# Carregar dados
data = pd.read_csv('data_/TSLA.csv')
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-09,22.366667,22.900000,22.098000,22.568666,22.568666,81493500
1,2019-01-10,22.293333,23.025999,22.119333,22.997999,22.997999,90846000
2,2019-01-11,22.806000,23.227333,22.584667,23.150667,23.150667,75586500
3,2019-01-14,22.825333,22.833332,22.266666,22.293333,22.293333,78709500
4,2019-01-15,22.333332,23.253332,22.299999,22.962000,22.962000,90849000


In [5]:
# Usar fechamento, máxima e mínima das ações
input_data = data[['Close', 'High', 'Low']].values

In [6]:
# Normalizar dados
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(input_data)

In [ ]:
# Criar sequências temporais
def create_sequences(data, seq_length):
    sequences = []SEQ_LENGTH = 10

In [ ]:
X = create_sequences(normalized_data, SEQ_LENGTH)

# Construir o autoencoder
model = Sequential()
for i in range(len(data) - seq_length):
    seq = data[i:i+seq_length]
    sequences.append(seq)
return np.array(sequences)

# Camada de encoder
model.add(LSTM(100, activation='relu', input_shape=(SEQ_LENGTH, 3), return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=False))
model.add(RepeatVector(SEQ_LENGTH))

# Camada de decoder
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(3)))

model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Treinar o autoencoder
model.fit(X, X, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

# Usar o encoder para obter a representação comprimida dos dados
encoder = Model(inputs=model.inputs, outputs=model.layers[1].output)
encoded_data = encoder.predict(X)